In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import time
from tqdm import tqdm

In [2]:
from selenium import webdriver

In [9]:
def musinsa_rank():
    url = f"https://www.musinsa.com/ranking/best?period=month&age=ALL&mainCategory=001&subCategory=&leafCategory=&price=&golf=false&kids=false&newProduct=false&exclusive=false&discount=false&soldOut=false&page=1&viewType=small&priceMin=&priceMax="
    response = requests.get(url)
    html = bs(response.text, 'lxml')
    musinsa_rank_df = rbnl(html)

    return musinsa_rank_df

In [10]:
def rbnl(html):
    musinsa_rank_df = pd.DataFrame()

    #순위 뽑기
    rank_html = html.select('#goodsRankList > li > p')
    rank_no_list = []

    for i in rank_html:
        rank_no_list.append(i.string.strip())

    musinsa_rank_df['순위'] = rank_no_list

    #브랜드 이름 뽑기
    brand_html = html.select('#goodsRankList > li > div.li_inner > div.article_info > p.item_title > a')
    brand_list = []

    for i in brand_html:
        brand_list.append(i.string)

    musinsa_rank_df['브랜드명']=brand_list

    #링크와 의류명 뽑기
    link_name_html = html.select('#goodsRankList > li > div.li_inner > div.article_info > p.list_info > a')
    link_list = []
    name_list = []

    for i in link_name_html:
        link_list.append(i['href'])
        name_list.append(i['title'])

    musinsa_rank_df['의류명']=name_list
    musinsa_rank_df['링크']=link_list

    #상세 페이지 크롤링
    musinsa_rank_df2 = specific_info(link_list)

    #데이터 프레임 옆으로 합치기
    musinsa_rank_df = pd.concat([musinsa_rank_df, musinsa_rank_df2], axis=1)

    return musinsa_rank_df

In [11]:
from pyasn1.compat.octets import null


def specific_info(link_list):
    headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.67 Safari/537.36'}

    musinsa_rank_df = pd.DataFrame()
    part_num_list=[]
    sex_list=[]
    view_list=[]
    sales_list=[]
    like_list=[]
    review_list=[]


    for link in tqdm(link_list):
        #         print(link)
        response_1=requests.get(link,headers=headers)
        html_1=bs(response_1.text, 'lxml')

        #품번 리스트 생성
        part_num_html=html_1.select('#product_order_info > div.explan_product.product_info_section > ul > li:nth-child(1) > p.product_article_contents > strong')
        part_num=part_num_html[0].get_text().split('/')[-1].strip()
        part_num_list.append(part_num)

        #성별 리스트 생성
        sex_html=html_1.select("#product_order_info > div.explan_product.product_info_section > ul > li > p.product_article_contents > span.txt_gender")
        sex=sex_html[0].get_text().replace('\n',' ').strip()
        sex_list.append(sex)

        #셀레니움으로 원하는 데이터 가져오기
        driver = webdriver.Chrome('chromedriver.exe')
        driver.get(link)
        sel_html=driver.page_source
        html_2=bs(sel_html)

        #조회수 가져오기
        view_html=html_2.find_all("strong", {"id":"pageview_1m"})
        view=view_html[0].get_text()
        view_list.append(view)

        #누적 판매 가져오기
        sales_html=html_2.find_all("strong", {"id":"sales_1y_qty"})
        sales=sales_html[0].get_text()
        sales_list.append(sales)

        #좋아요 수 가져오기
        like_html=html_2.find_all("span", {"class": "prd_like_cnt"})
        like=like_html[0].get_text()
        like_list.append(like)

        #구매 후기 개수 가져오기
        review_html=html_2.find_all("span", {"class": "prd-score__review-count"})
        if len(review_html) == 0:
            review = "후기 0개"
        else:
            review=review_html[0].get_text()
        review_list.append(review)


        #드라이버 닫아주기
        driver.close()

        #시간 추가
        time.sleep(0.01)

    musinsa_rank_df['품번']=part_num_list
    musinsa_rank_df['성별']=sex_list
    musinsa_rank_df['조회수']=view_list
    musinsa_rank_df['누적판매량(1년)']=sales_list
    musinsa_rank_df['좋아요']=like_list
    musinsa_rank_df['구매후기 개수']=review_list

    return musinsa_rank_df

In [12]:

final_df=musinsa_rank()

  0%|          | 0/90 [00:00<?, ?it/s]C:\Users\jpc10\AppData\Local\Temp\ipykernel_30204\3982278999.py:32: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('chromedriver.exe')
  0%|          | 0/90 [00:12<?, ?it/s]


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=111.0.5563.65)
Stacktrace:
Backtrace:
	(No symbol) [0x0093DCE3]
	(No symbol) [0x008D39D1]
	(No symbol) [0x007E4DA8]
	(No symbol) [0x007CD0D3]
	(No symbol) [0x0082EA8B]
	(No symbol) [0x0083D093]
	(No symbol) [0x0082ACC6]
	(No symbol) [0x00806F68]
	(No symbol) [0x008080CD]
	GetHandleVerifier [0x00BB3832+2506274]
	GetHandleVerifier [0x00BE9794+2727300]
	GetHandleVerifier [0x00BEE36C+2746716]
	GetHandleVerifier [0x009E6690+617600]
	(No symbol) [0x008DC712]
	(No symbol) [0x008E1FF8]
	(No symbol) [0x008E20DB]
	(No symbol) [0x008EC63B]
	BaseThreadInitThunk [0x775F7D69+25]
	RtlInitializeExceptionChain [0x77A4B74B+107]
	RtlClearBits [0x77A4B6CF+191]


In [8]:
file_name = f"musinsa_ranking_category_page.csv"
final_df.to_csv(file_name, index=False)
pd.read_csv(file_name)

,순위,브랜드명,의류명,링크,품번,성별,조회수,누적판매량(1년),좋아요,구매후기 개수
0,1위,수아레,워셔블 하찌 니트 - 11 COLOR,https://www.musinsa.com/app/goods/2396645?loc=...,1709399,남,17.4만 회 이상,9.2만 개 이상,"82,193","후기 22,446개 보기"
1,2위,무신사 스탠다드,레이어드 크루 넥 반팔 티셔츠_일반 기장 [화이트],https://www.musinsa.com/app/goods/2086653?loc=...,MMATS701-WH,남,19.2만 회 이상,21.6만 개 이상,"74,286","후기 37,692개 보기"
2,3위,스파오,(시티보이) 오버핏 옥스포드 셔츠_SPYWD23C01,https://www.musinsa.com/app/goods/2272830?loc=...,SPYWD23C01,남,22.5만 회 이상,5만 개 이상,"45,533","후기 17,512개 보기"
3,4위,예일,(23SS) 2 TONE ARCH HOODIE GRAY,https://www.musinsa.com/app/goods/2326935?loc=...,YD01HD1007GR,남 여,14.2만 회 이상,6.7만 개 이상,"83,194","후기 12,586개 보기"
4,5위,토피,에센셜 후드 스웻 셔츠 (MELANGE GREY),https://www.musinsa.com/app/goods/2314616?loc=...,TAS-EHSS,남,12.7만 회 이상,4.1만 개 이상,"42,672","후기 7,724개 보기"
...,...,...,...,...,...,...,...,...,...,...
85,86위,무신사 스탠다드,베이식 긴팔 티셔츠 [화이트],https://www.musinsa.com/app/goods/1144989?loc=...,MITS0009-WH,남,4.1만 회 이상,4.8만 개 이상,"58,709","후기 66,737개 보기"
86,87위,브라운브레스,TAG HOODIE - BURGUNDY,https://www.musinsa.com/app/goods/2722805?loc=...,BISMHDB01BD,남 여,2.9만 회 이상,3.8천 개 이상,"12,870",후기 839개 보기
87,88위,수아레,[리뉴얼] 하프 터틀넥 니트 세트,https://www.musinsa.com/app/goods/1558847?loc=...,670743,남,2.9만 회 이상,4.8만 개 이상,"49,898","후기 27,205개 보기"
88,89위,인템포무드,해비 코튼 썸홀 후디 : 15color,https://www.musinsa.com/app/goods/2319988?loc=...,487389,남,3.1만 회 이상,9.1천 개 이상,"17,900","후기 5,286개 보기"
